In [1]:
!pip install opencv-python matplotlib imageio tensorflow

In [2]:
import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio

2024-03-07 11:55:19.281816: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 11:55:19.372397: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 11:55:19.376212: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mathildaweston/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/cv2/../../

In [18]:
# speakers = ['s1']

# paths_to_clean = ['/'.join([os.getcwd(),'data/alignments', speaker]) for speaker in speakers]
# # path_to_clean

'/home/mathildaweston/code/mathildaweston/lip-reading-plan/data/alignments/s1'

In [24]:
speakers = 's1'

path_to_clean = ['/'.join([os.getcwd(),'data/alignments', speaker])]
path_to_clean

['/home/mathildaweston/code/mathildaweston/lip-reading-plan/data/alignments/s1']

In [48]:
file_names = []
for folder_path in paths_to_clean:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.align'):
                file_name = file.replace(".align", "")
                file_names.append(file_name)
# len(file_names)

In [26]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

In [27]:
def load_alignments(path:str) -> List[str]: 
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil': #Removing the silences
            tokens = [*tokens,' ',line[2]]
    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [37]:
def load_align_data(file_name: str): 
    alignment_path = os.path.join(path_to_clean[0], f'{file_name}.align')
    alignments = load_alignments(alignment_path).numpy().tolist()

    return alignments

In [47]:
# len(load_align_data('srah7a'))

In [45]:
def make_dict(file_names):
    align_dict = dict(zip(file_names, [load_align_data(file_name) for file_name in file_names]))
    return align_dict

In [49]:
# make_dict(file_names)

In [50]:
s1_aligns = make_dict(file_names)

In [58]:
import json

with open('/'.join([os.getcwd(), 'data/processed_alignments', speaker, f'{speaker}_proc_align.json']), 'w') as fp:
    json.dump(s1_aligns, fp)

In [61]:
f = open('/'.join([os.getcwd(), 'data/processed_alignments', speaker, f'{speaker}_proc_align.json']))

test_align = json.load(f)

In [63]:
# test_align